# Imports

In [3]:
print('hello')

hello


In [4]:
import pandas as pd
import numpy as np
# import nltk
# nltk.download('punkt')
from collections import Counter
# import fasttext.util
import os
import re
from collections import Counter
from tqdm.auto import tqdm
tqdm.pandas()
# fasttext.util.download_model('en', if_exists='ignore')

# TF IDF

In [2]:
# get generic word freqs
# for each doc get word freqs
# calculate top doc word freq / generic word freq

In [6]:
# ! pip install fasttext-wheel

In [4]:
df = pd.read_excel('data/SUBTLEXusfrequencyabove1.xls')

In [6]:
df = df.set_index('Word')

In [7]:
freqs = df['FREQcount']

In [8]:
freqs['the']

1501908

In [8]:
with open('data/Thoughts_en/2nd law.9ef1ed84168a5c49d78452e206e7ba04dd473cd314d975cadab164258650b384.docx') as f:
    data = f.read()


In [9]:
words = nltk.word_tokenize(data.lower())

In [10]:
cnt = Counter(words)

In [12]:
tf_idf = {c: n/(freqs[c] if c in freqs else 1) for c,n in cnt.items()}

In [13]:
l = sorted([(c,w) for w,c in tf_idf.items()], reverse=True)

In [15]:
sum([f for f,w in l])

36.26542819941986

In [25]:
# fasttext.util.download_model('en', if_exists='ignore', dimension=100)

In [17]:
# import io

# def load_vectors(fname):
#     fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
#     n, d = map(int, fin.readline().split())
#     data = {}
#     for line in fin:
#         tokens = line.rstrip().split(' ')
#         data[tokens[0]] = map(float, tokens[1:])
#     return data

In [18]:
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip -P data/

In [19]:
# ! unzip data/wiki-news-300d-1M.vec.zip

In [20]:
# ft = load_vectors('data/wiki-news-300d-1M.vec')

In [ ]:
# ! pip install -U sister
# fails due to c++ compiler mismatch

In [7]:
#  ! pip install -U sentence-transformers --no-cache-dir

In [16]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [19]:
sentences = data.split('\n')

In [20]:
sentence_embeddings = model.encode(sentences)

In [27]:
sentence_embeddings = np.ndarray.tolist(sentence_embeddings)

In [111]:
docs = []
embeddings = []
dirname = "data/Thoughts_en"
for doc in tqdm(os.listdir(dirname)):
    if not doc.endswith('.docx'):
        continue
    with open(os.path.join(dirname, doc)) as f:
        data = f.read()
    sentences = data.split('\n')
    sentence_embeddings = model.encode(sentences)
    docs += sentences
    embeddings += np.ndarray.tolist(sentence_embeddings)


In [112]:
embeddings_df = pd.DataFrame(embeddings)

In [146]:
embeddings_df.to_csv('data/Thoughts_en.vec.tsv', index=False, header=False, sep='\t')

In [114]:
docs = [re.sub('\t', '    ', doc) for doc in docs]

In [35]:
with open('data/Thoughts_en.txt.tsv', 'w') as f:
    f.write('\n'.join(docs))

In [116]:
all_docs = '\n'.join(docs)

In [117]:
cnt = Counter(all_docs)

In [118]:
cnt = sorted(cnt.items(), key=lambda x: x[1])

In [120]:
low_cnt = [c for c in cnt if c[1]<=10]

In [121]:
for c in low_cnt:
    all_docs = re.sub(c[0], '_', all_docs)

In [74]:
# i = all_docs.index(cnt[6][0])

In [75]:
# all_docs[i-10:i+10]

In [122]:
docs = all_docs.split('\n')

In [124]:
docs_df = pd.DataFrame(docs)

In [130]:
docs_df['action'] = docs_df[0].apply(lambda x: 'need' in x)

In [139]:
docs_df.columns = ['doc', 'is_action']

In [141]:
docs_df.to_csv('data/Thoughts_en.doc.tsv', index=False, sep='\t')